In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn import preprocessing
from GGLasso.gglasso.problem import glasso_problem
from utils import transform_features, scale_array_by_diagonal
from utils import PCA

# Data

## Raw data

In [ ]:
raw = pd.read_csv('data/composition_feature-table.tsv', sep='\t', index_col = 0)

print(raw.shape)
raw.head()

In [ ]:
# raw.to_csv("data/count_table.csv", index=True)
raw.iloc[:, 0].plot.hist(bins=24, alpha=1).get_figure().savefig('plots/raw_count.png')

## CLR

In [ ]:
clr = transform_features(raw, transformation="clr")
clr.head()

In [ ]:
clr.iloc[:, 0].plot.hist(bins=24, alpha=1).get_figure().savefig('plots/clr_count.png')

## Modified CLR

In [ ]:
mclr = transform_features(raw, transformation="mclr")
mclr.head()

In [ ]:
mclr.iloc[:, 0].plot.hist(bins=24, alpha=1).get_figure().savefig('plots/mclr_count.png')

## Covariates

In [ ]:
# covariates
meta = pd.read_csv('data/acm_meta.tsv', sep='\t', index_col = 0)

# select only numeric features
meta = meta.loc[:, meta.iloc[0, :] != 'categorical']
meta = meta.apply(pd.to_numeric, errors='coerce')

# drop QIIME2 header
meta = meta.iloc[1:]
# fill missing values with zeros
meta = meta.fillna(0)

print(meta.shape)
meta.head()

In [ ]:
fig, axis = plt.subplots(5,3,figsize=(15, 20))
meta.hist(ax=axis)

# fig.savefig('plots/meta_unscaled.png')

## Scale covariates

In [ ]:
scaler = preprocessing.StandardScaler().fit(meta)
meta_scaled = scaler.transform(meta)

meta_scaled = pd.DataFrame(meta_scaled, index=meta.index, columns=meta.columns)
meta_scaled.head()

In [ ]:
fig, axis = plt.subplots(5,3,figsize=(15, 20))
meta_scaled.hist(ax=axis)

# fig.savefig('plots/meta_scaled.png')

## Merge data with covariates

In [ ]:
# # transpose count data
# mclr_T = mclr.T
# # join by sample id
# df = mclr_T.join(meta_scaled)

# transpose count data
clr_T = clr.T
# join by sample id
df = clr_T.join(meta_scaled)

#check if any missing values
print("Any missing values?: {0}".format(df.isnull().sum().any()))
print(df.shape)
df.head()

In [ ]:
# Rename long feature IDs with concise names
vis_df = df.copy()
id_dict = dict()

i = 1
for col in vis_df.columns:
    # length of ASVs identifier
    if len(col) == 32:
        asv_name = "ASV_{0}".format(i)
        id_dict[asv_name] = col
        vis_df.rename(columns={col: asv_name}, inplace=True)
        
        i += 1

In [ ]:
vis_df.head()

## Covariance

In [ ]:
n_cov = meta_scaled.shape[1]
asv = df.iloc[:, :-n_cov]
S = np.cov(asv.T.values, bias=True)

# correlation between ASVs ONLY
corr = scale_array_by_diagonal(S)

asv_names = vis_df.iloc[:, :-n_cov].columns
vis_S = pd.DataFrame(corr, columns=asv_names, index=asv_names)
print(vis_S.shape)
vis_S.head()

In [ ]:
fig = px.imshow(vis_S, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 1000, height = 1000,
                 title='Correlation: ASVs', title_x=0.5)

# fig.write_image("plots/asv_correlation.png")

In [ ]:
S_meta = np.cov(df.T.values, bias=True)

# correlation between ASVs and covariates
corr_meta = scale_array_by_diagonal(S_meta)

vis_S_meta = pd.DataFrame(corr_meta, columns=vis_df.columns, index=vis_df.columns)
print(vis_S_meta.shape)
vis_S_meta.head()

In [ ]:
fig = px.imshow(vis_S_meta, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 1000, height = 1000,
                 title='Covariance: ASVs and covariates', title_x=0.5)

# fig.write_image("plots/asv_and_covariates_correlation.png")

# Models

In [ ]:
N = asv.shape[0]
p = asv.shape[1]
print("Shape of data without covariates: {0}, {1}".format(N, p))

N_meta = df.shape[0]
p_meta = df.shape[1]
print("Shape of data with covariates: {0}, {1}".format(N_meta, p_meta))

**Statement 1**:

We know that environment has a large impact on microbial composition.
We would like to investigate this effect and run single graphical lasso model while accounting for latent components.

## SGL + low-rank

In [ ]:
lambda1_range = np.logspace(0, -4, 15)
mu1_range = np.logspace(2, -1, 10)

modelselect_params = {'lambda1_range': lambda1_range, 'mu1_range': mu1_range}

In [ ]:
P_SGL_low = glasso_problem(corr, N, latent=True, do_scaling=False)
print(P_SGL_low)

In [ ]:
P_SGL_low.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.1)
print(P_SGL_low.reg_params)

In [ ]:
for stat in ['AIC', 'SP', 'RANK', 'LAMBDA', 'MU', 'BEST']:
    print(stat)
    print(P_SGL_low.modelselect_stats[stat])

In [ ]:
precision_SGL_low = pd.DataFrame(P_SGL_low.solution.precision_, columns=asv_names, index=asv_names)

fig = px.imshow(-1*precision_SGL_low, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 1000, height = 1000,
                 title='Estimated inverse covariance: ASVs', title_x=0.5)

# fig.write_image("plots/sgl_asv_only.png")

In [ ]:
L = pd.DataFrame(P_SGL_low.solution.lowrank_, columns=asv_names, index=asv_names)

print("Rank: {0}".format(np.linalg.matrix_rank(L)))

<!-- **Statement 2**:

According to the models selection statistics, the chosen optimal hyperparameters lay in the middle of the grid and we have reached a global minimum of our function. Yet we can try to find such hyperparameters which give us low-rank solutions with the rank of our interest, e.g., from 0 to 10 as well as desired data sparsity level. -->

In [ ]:
# lambda1_range = [0.07]
# mu1_range = [1.8, 1.55, 1.4, 1.2, 1.18, 1.1, 1.09, 1.05, 1.015, 1]

# modelselect_params = {'lambda1_range': lambda1_range, 'mu1_range': mu1_range}

In [ ]:
# P_SGL_low_tuned = glasso_problem(corr, N, latent=True, do_scaling=False)
# print(P_SGL_low_tuned)

In [ ]:
# P_SGL_low_tuned.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.1)
# print(P_SGL_low_tuned.reg_params)

In [ ]:
# for stat in ['AIC', 'SP', 'RANK', 'MU', 'BEST']:
#     print(stat)
#     print(P_SGL_low_tuned.modelselect_stats[stat])

In [ ]:
# L_tuned = pd.DataFrame(P_SGL_low_tuned.solution.lowrank_, columns=asv_names, index=asv_names)

# rank = np.linalg.matrix_rank(L_tuned)
# print("Rank: {0}".format(rank))

In [ ]:
# fig = px.imshow(L_tuned, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
# fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 1000, height = 1000,
#                  title='Low-rank: ASVs', title_x=0.5)

# fig.write_image("plots/low_rank_asv_only.png")

**Statement 3**:

Imagine we don't have covariate information.

The Graphical Lasso solution is of the form Θ−𝐿 where Θ is sparse and 𝐿 has low rank. We use the low rank component of the Graphical Lasso solution in order to do a robust PCA. For this, we use the eigendecomposition 𝐿=𝑉Σ𝑉𝑇 where the columns of 𝑉 are the orthonormal eigenvecors and Σ is diagonal containing the eigenvalues. Denote the columns of 𝑉 corresponding only to positive eigenvalues with 𝑉̃ ∈ℝ𝑝×𝑟 and Σ̃ ∈ℝ𝑟×𝑟 accordingly, where 𝑟=rank(𝐿). Then we have 𝐿=𝑉̃ Σ̃ 𝑉̃ 𝑇.
Now we project the data matrix 𝑋∈ℝ𝑝×𝑁 onto the eigenspaces of 𝐿−1 - which are the same as of 𝐿 - by computing:

In [ ]:
# proj, loadings, eigv = PCA(asv, L_tuned, inverse=True)
proj, loadings, eigv = PCA(asv, L, inverse=True)

In [ ]:
depth = raw.sum(axis=0)
ph = vis_df["ph"]

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

fig, ax = plt.subplots(1,1)
im = ax.scatter(proj[:,0], ph, c = depth, cmap = plt.cm.Blues, vmin = 0)
cbar = fig.colorbar(im)
cbar.set_label("Sampling depth")
ax.set_xlabel(f"PCA component 1 with eigenvalue {eigv[0]}")
ax.set_ylabel("pH")

print("Spearman correlation between pH and 1st component: {0}, p-value: {1}".format(stats.spearmanr(ph, proj[:,0])[0],
                                                                              stats.spearmanr(ph, proj[:,0])[1]))


**Statement 4**:

Alternatively, we could solve SGL with adaptive penalization and covariance matrix where covariates are included. So, we could see if loading vectors of PCA we found with the previous model where was no information about covariates are correlated with the solution where we include covariate information. Thus, we could argue that SGL model with low-rank component can capture the information which is unknown but important for microbial composition.

## SGL with covariates and adaptive penalization procedure

In [ ]:
# create lambda matrix full of zeros
shape_meta = (p_meta, p_meta)
mask = np.zeros(shape_meta)

# add small constant, so ADMM could converge
mask = mask + 0.01

# heavy penalize species
n_bugs = len(asv.columns)
bugs_block = np.ones((n_bugs, n_bugs))
mask[0:n_bugs, 0:n_bugs] += bugs_block - 0.01
lambda1_mask_exp = mask

In [ ]:
lambda1_mask_exp.shape

In [ ]:
df_mask_exp = pd.DataFrame(lambda1_mask_exp, columns=vis_df.columns, index=vis_df.columns)

fig = px.imshow(df_mask_exp, color_continuous_scale='RdBu_r')
fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 1000, height = 1000,
                 title='Lambda-mask matrix: weights before the penalization term', title_x=0.5)

fig.add_annotation(text="$\lambda=1$",
                  xref="paper", yref="paper", font=dict(color='yellow',size=155),
                  x=0.5, y=0.5, showarrow=False)
fig.add_annotation(text="$\lambda=0.01$",
                  xref="paper", yref="paper", font=dict(color='yellow',size=155),
                  x=0.5, y=0.05, showarrow=False)

fig.update_coloraxes(showscale=False)

# fig.write_image("plots/lambda_mask.png")

In [ ]:
### the same default grid, no tuning
lambda1_range = np.logspace(0, -4, 15)

### less sparsity in ASVs block
# lambda1_range = np.logspace(-0.5, -1, 15)

modelselect_params = {'lambda1_range': lambda1_range, 'lambda1_mask': lambda1_mask_exp}

In [ ]:
P_SGL_adapt = glasso_problem(corr_meta, N_meta, latent=False, do_scaling=False)
print(P_SGL_adapt)

In [ ]:
P_SGL_adapt.model_selection(modelselect_params=modelselect_params, method='eBIC', gamma=0.1)
print(P_SGL_adapt.reg_params)

Note: mu1=0, i.e., no low-rank

In [ ]:
for stat in ['AIC', 'SP', 'LAMBDA', 'BEST']:
    print(stat)
    print(P_SGL_adapt.modelselect_stats[stat])

In [ ]:
precision_adapt = pd.DataFrame(P_SGL_adapt.solution.precision_, columns=vis_df.columns, index=vis_df.columns)
precision_adapt.head()

In [ ]:
fig = px.imshow(-1*precision_adapt, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 1000, height = 1000,
                 title='Esatimated inverse covariance (adaptive)', title_x=0.5)
# fig.write_image("plots/sgl_asv_and_cov.png")

In [ ]:
inv_cov = precision_adapt.iloc[:-n_cov, -n_cov:]

fig = px.imshow(inv_cov, color_continuous_scale='RdBu_r', zmin=-1, zmax=1)
fig.update_layout(margin = dict(t=100,r=100,b=100,l=100), width = 600, height = 3000,
                 title='Negative inverse covariance between ASVs and covariates', title_x=0.5)

# fig.write_image("plots/sgl_cov.png")

In [ ]:
rank = loadings.shape[1]

In [ ]:
# fig, ax = plt.subplots(3, 15, figsize=(200, 50))

# j = 0
# for col in inv_cov.columns:

#     for i in range(0, rank):
#         spearman_r = stats.spearmanr(inv_cov[col], loadings[:, i])
#         if abs(spearman_r[0]) > 0.4:
#             print("Spearman correlation between estimated pH and {0}th principal axis: {1}, \n\t p-value: {2}".format(
#             axis, spearman_r[0], spearman_r[1]))
            
        
#         ax[i][j].scatter(loadings[:,i], inv_cov[col])
#         ax[i][j].set_xlabel("Principal axis (loading vector) {0}".format(i))
#         ax[i][j].set_ylabel("{0}".format(col))

        
#     j += 1
    
# plt.savefig("plots/big_scatter.png")

**Statement 5**:

Now we would like to test if the latent components of our solution can explain the covariates.

On the left-hand subplot, we project estimated covariates to the low-rank solution.
On the right-hand subplot, we project original covariates to the same low-rank solution where no covariates were included.

In [ ]:
# plot PCs vs covariates

eigv_sum = np.sum(eigv)
var_exp = [(value / eigv_sum) for value in sorted(eigv, reverse=True)]

x = raw.sum(axis=0)
seq_depth = pd.DataFrame(data=x, columns=["sequencing depth"])

test_df = vis_df.copy()
test_df = test_df.join(seq_depth)

for col in meta_scaled.columns:

    for i in range(0, rank):

        r_2 = stats.spearmanr(test_df[col], proj[:, i])[0]
        p_value = stats.spearmanr(test_df[col], proj[:, i])[1]
        
        if abs(r_2) > 0.6:
            x = loadings[:,i]
            y = inv_cov[col]
            
            # Find the slope and intercept of the best fit line
            slope, intercept = np.polyfit(x, y, 1)

            # Create a list of values in the best fit line
            abline_values = [slope * i + intercept for i in x]
            
            fig, ax = plt.subplots(nrows = 1, ncols = 2, sharex=False, sharey = False, squeeze=False, figsize=(15, 7))
            ax[0][0].scatter(x, y)
            ax[0][0].plot(x, abline_values, 'b')
            ax[0][0].set_xlabel("Principal axis (loading vector) {0}".format(i+1))
            ax[0][0].set_ylabel("estimated {0}".format(col))
            
            spearman_corr = stats.spearmanr(test_df[col], proj[:, i])[0]
            p_value = stats.spearmanr(test_df[col], proj[:, i])[1]

            title = "Spearman correlation {0}".format(np.round(spearman_corr, 3))
            im = ax[0][1].scatter(proj[:, i], test_df[col], c=test_df['sequencing depth'], cmap=plt.cm.Blues, vmin=0)
            cbar = fig.colorbar(im)


            cbar.set_label("Sampling depth")
            ax[0][1].set_xlabel("PC{0} ({1}%)".format(i + 1, str(100 * var_exp[i])[:4]))
            ax[0][1].set_ylabel("{0}".format(col))
            ax[0][1].title.set_text(title)
            
            plt.savefig("plots/pc_plots/scatter_pc_{0}_{1}.png".format(i, col))